In [1]:
import numpy as np
import pandas as pd
import os
import glob
import collections
import json
import datetime
from bs4 import BeautifulSoup
import urllib

In [2]:
# # google colab only
# # After mounting google drive
# %cd drive/MyDrive/Colab\ Notebooks/dbdiff
# %ls modules
# !pip install js2py

In [3]:
import re
from modules import notes, utils
import js2py
import pdb

In [4]:
def get_scores_list(titletbl, actbl, playmode="dp"):
    ret_list = []
    for fbasename in list(titletbl):
        diffs = {}
        acin = {}
        ver, numNotes, opt, genre, arist, title, *_ = titletbl[fbasename]
        try:
            actbl_list = list(actbl[fbasename])
            subtitle = "" if len(actbl_list) <= 23 else actbl_list[23] 
            if playmode == "dp":
                acin["n"], acin["h"], acin["a"], acin["l"] = actbl_list[16:23:2] if actbl_list[0]&1!=0 else [False for x in "nhal"]
                diffs["n"], diffs["h"], diffs["a"], diffs["l"] = actbl_list[15:22:2]
            elif playmode == "sp":
                acin["n"], acin["h"], acin["a"], acin["l"] = actbl_list[6:13:2] if actbl_list[0]&1!=0 else [False for x in "nhal"]
                diffs["n"], diffs["h"], diffs["a"], diffs["l"] = actbl_list[5:12:2]
        except:
            continue
        for_add = [[fbasename, diffs[x], x, ver, bool(acin[x]&4), subtitle] for x in "nhal" if diffs[x] > 0]
        ret_list.extend(for_add)
    return ret_list

tables = ["https://textage.cc/score/actbl.js", "https://textage.cc/score/titletbl.js"]
save_names = ["actbl.js", "titletbl.js"]

for url, save_name in zip(tables, save_names):
    urllib.request.urlretrieve(url, save_name)

titletbl, actbl = utils.get_tables("titletbl.js", "actbl.js")
scores = get_scores_list(titletbl, actbl, playmode="dp")

In [5]:
adjoin_key_idx = 3 # 1~7
mucha_sec = 60 / 190 / 4 # sec: BPM190の16分
keyNum = 7

def is_adjoin(adjoin_sp_list, playside):
    isAdjoin, isMuri = False, False
    for on_sc_list in adjoin_sp_list:
        if (playside == 1 and not all(x <= (adjoin_key_idx-1) for x in on_sc_list[0])) or \
           (playside == 2 and not all(x >= (keyNum-adjoin_key_idx-1) for x in on_sc_list[0])):
            isMuri = True
        elif (playside == 1 and not all(x > (adjoin_key_idx-1) for x in on_sc_list[0])) or \
             (playside == 2 and not all(x < (keyNum-adjoin_key_idx-1) for x in on_sc_list[0])):
            isAdjoin = True
        if isMuri and isAdjoin:
            break
    return isAdjoin, isMuri

def is_mucha(arr, sc_arr_sp, arr_bpm, playside):
    st, en = (1, 8) if playside == 1 else (8, -1)
    for idx in np.array(np.where(sc_arr_sp==1))[0]:
        bpm = arr_bpm[idx]
        range_dot = int(int(bpm) * mucha_sec * 4 / 5)
        if (2 <= arr[st:en, idx-range_dot:idx+range_dot+1, :].sum(axis=(0, 2)).astype(bool).astype(int).sum()):
            return True
    return False


def check_adjoin(arr, arr_bpm):
    sc_arr_1p = (arr[0,:,:] == 1).sum(axis=-1).astype(bool)
    sc_arr_2p = (arr[-1,:,:] == 1).sum(axis=-1).astype(bool)
    isMucha = is_mucha(arr, sc_arr_1p, arr_bpm, 1) or is_mucha(arr, sc_arr_2p, arr_bpm, 2)
    sc_on_1p = arr[1:8, sc_arr_1p, :].sum(axis=2) # 7 x scの枚数
    sc_on_2p = arr[8:-1, sc_arr_2p, :].sum(axis=2)
    hasAdjoin_1p, hasAdjoin_2p = sc_on_1p.sum() > 0, sc_on_2p.sum() > 0
    if not (hasAdjoin_1p or hasAdjoin_2p):
        if isMucha:
            return True, {"1p":{}, "2p":{}}, "無茶皿"
        else:
            return False, {}, None
    adjoin_1p_list = [np.array(np.where(x==1)).tolist() for x in sc_on_1p.transpose()]
    adjoin_2p_list = [np.array(np.where(x==1)).tolist() for x in sc_on_2p.transpose()]
    isAdjoin, isMuri = [x or y for x, y in zip(is_adjoin(adjoin_1p_list, 1), is_adjoin(adjoin_2p_list, 2))]
    judge_list = ["無茶皿", "隣接皿", "無理皿"]
    judge_bool_list = [isMucha, isAdjoin, isMuri]
    judge = ", ".join([judge_list[idx] for idx, x in enumerate(judge_bool_list) if x])
    adjoin_1p = collections.Counter(list(map(lambda x:",".join([str(y+1) for y in x[0]]), adjoin_1p_list)))
    adjoin_2p = collections.Counter(list(map(lambda x:",".join([str(y+1) for y in x[0]]), adjoin_2p_list)))
    del adjoin_1p[""]
    del adjoin_2p[""]
    return True, {"1p":dict(adjoin_1p), "2p":dict(adjoin_2p)}, judge


In [5]:
pp = "source/SCORE_OFF/20/_casino.html"
nt = notes.Notes(pp)
nt.convertNotes()
hasAdjoin, adjoins, judge = check_adjoin(nt.notesArray, nt.bpmArray)

In [6]:
judge

'無茶皿, 無理皿'

In [6]:
adjoin_scores = []
faild_name = []
for score in scores[:]:
    fbasename, lv, diff_char, ver, acin, subtitle = score
    parDir = os.path.join("..", "dbdiff", "source", "SCORE_OFF")
    ver = "s" if ver == "SS" or ver == 35 else str(ver)
    fname = os.path.join(parDir, ver, fbasename+".html")
    try:
        nt = notes.Notes(fname, difficulty=diff_char)
        print("title: {0}, diff: {1}...".format(nt.title, nt.difficulty), end="")
        nt.convertNotes()
        hasAdjoin, adjoins, judge = check_adjoin(nt.notesArray, nt.bpmArray)
        if hasAdjoin:
            print("has adjoin !!!")
            adjoin_scores.append({"title":nt.title, "level":lv, "diff":diff_char, "adjoin_keys":adjoins, "judge":judge, "acin":acin, "subtitle":subtitle, "ver":ver, "fbasename":fbasename})
        elif not judge:
            print("no adjoin")
    except:
        faild_name.append(fname)
        continue


title: 10000 MILES AWAY, diff: n...no adjoin
title: 10000 MILES AWAY, diff: h...no adjoin
title: 10000 MILES AWAY, diff: a...no adjoin
title: 1001 -ESCAPE-, diff: n...no adjoin
title: 1001 -ESCAPE-, diff: h...no adjoin
title: 1001 -ESCAPE-, diff: a...no adjoin
title: 100% minimoo-G, diff: n...no adjoin
title: 100% minimoo-G, diff: h...no adjoin
title: 100% minimoo-G, diff: a...has adjoin !!!
title: 1116, diff: n...no adjoin
title: 1116, diff: h...no adjoin
title: 1116, diff: a...no adjoin
title: 1989, diff: n...no adjoin
title: 1989, diff: h...no adjoin
title: 199024club -Re:BounceKiller-, diff: n...no adjoin
title: 199024club -Re:BounceKiller-, diff: h...no adjoin
title: 199024club -Re:BounceKiller-, diff: a...has adjoin !!!
title: 19,November, diff: n...has adjoin !!!
title: 19,November, diff: h...has adjoin !!!
title: 19,November, diff: a...has adjoin !!!
title: 1st Samurai, diff: n...no adjoin
title: 1st Samurai, diff: h...no adjoin
title: 1st Samurai, diff: a...no adjoin
title: 20

title: バビロニア, diff: n...no adjoin
title: バビロニア, diff: h...no adjoin
title: バビロニア, diff: a...no adjoin
title: バッドエンド・シンドローム, diff: n...no adjoin
title: バッドエンド・シンドローム, diff: h...no adjoin
title: バッドエンド・シンドローム, diff: a...no adjoin
title: バッド・スイーツ、バッド・ドリーム, diff: n...no adjoin
title: バッド・スイーツ、バッド・ドリーム, diff: h...no adjoin
title: バッド・スイーツ、バッド・ドリーム, diff: a...no adjoin
title: 爆ゼニ☆がってんワーカー, diff: n...no adjoin
title: 爆ゼニ☆がってんワーカー, diff: h...no adjoin
title: 爆ゼニ☆がってんワーカー, diff: a...no adjoin
title: バンブーソード・ガール, diff: n...no adjoin
title: バンブーソード・ガール, diff: h...no adjoin
title: バンブーソード・ガール, diff: a...no adjoin
title: ベィスドロップ・フリークス, diff: n...no adjoin
title: ベィスドロップ・フリークス, diff: h...no adjoin
title: ベィスドロップ・フリークス, diff: a...no adjoin
title: ベィスドロップ・フリークス, diff: l...no adjoin
title: バスルーム ロマンス, diff: n...no adjoin
title: バスルーム ロマンス, diff: h...no adjoin
title: バスルーム ロマンス, diff: a...no adjoin
title: 紅牡丹, diff: n...no adjoin
title: 紅牡丹, diff: h...no adjoin
title: 紅牡丹, diff: a...no adjoin
title: 青の洞

title: 幻想系世界修復少女, diff: a...no adjoin
title: 《GRANDMASTER》, diff: n...no adjoin
title: 《GRANDMASTER》, diff: h...no adjoin
title: 《GRANDMASTER》, diff: a...no adjoin
title: グッバイ宣言, diff: n...no adjoin
title: グッバイ宣言, diff: h...no adjoin
title: グッバイ宣言, diff: a...no adjoin
title: グラナダの風, diff: n...no adjoin
title: グラナダの風, diff: h...no adjoin
title: グラナダの風, diff: a...no adjoin
title: 閠槞彁の願い, diff: n...no adjoin
title: 閠槞彁の願い, diff: h...no adjoin
title: 閠槞彁の願い, diff: a...no adjoin
title: ぐだふわエブリデー, diff: n...no adjoin
title: ぐだふわエブリデー, diff: h...no adjoin
title: ぐだふわエブリデー, diff: a...no adjoin
title: 紅蓮華, diff: n...no adjoin
title: 紅蓮華, diff: h...no adjoin
title: 紅蓮華, diff: a...no adjoin
title: ギョギョっと人魚&#9832;爆婚ブライダル, diff: n...no adjoin
title: ギョギョっと人魚&#9832;爆婚ブライダル, diff: h...no adjoin
title: ギョギョっと人魚&#9832;爆婚ブライダル, diff: a...no adjoin
title: 儚き恋の華, diff: n...no adjoin
title: 儚き恋の華, diff: h...no adjoin
title: 儚き恋の華, diff: a...no adjoin
title: 花吹雪 〜 IIDX LIMITED 〜, diff: n...no adjoin
title: 

title: 恋はどう？モロ◎波動OK☆方程式!!, diff: a...no adjoin
title: 恋は白帯、サンシロー, diff: n...no adjoin
title: 恋は白帯、サンシロー, diff: h...no adjoin
title: 恋は白帯、サンシロー, diff: a...no adjoin
title: 恋は白帯、サンシロー, diff: l...no adjoin
title: 恋する☆宇宙戦争っ!!, diff: n...no adjoin
title: 恋する☆宇宙戦争っ!!, diff: h...no adjoin
title: 恋する☆宇宙戦争っ!!, diff: a...no adjoin
title: 恋する☆宇宙戦争っ!!, diff: l...no adjoin
title: 高高度降下低高度開傘, diff: n...no adjoin
title: 高高度降下低高度開傘, diff: h...no adjoin
title: 高高度降下低高度開傘, diff: a...no adjoin
title: こっちを向いてよ, diff: n...has adjoin !!!
title: こっちを向いてよ, diff: h...has adjoin !!!
title: こっちを向いてよ, diff: a...has adjoin !!!
title: 今宵、ロマンス横丁。, diff: n...no adjoin
title: 今宵、ロマンス横丁。, diff: h...no adjoin
title: 今宵、ロマンス横丁。, diff: a...no adjoin
title: 黒紅掬い, diff: n...no adjoin
title: 黒紅掬い, diff: h...no adjoin
title: 黒紅掬い, diff: a...no adjoin
title: 黒髪乱れし修羅となりて, diff: n...no adjoin
title: 黒髪乱れし修羅となりて, diff: h...no adjoin
title: 黒髪乱れし修羅となりて, diff: a...no adjoin
title: ■□模様, diff: n...no adjoin
title: ■□模様, diff: h...no

title: 夏の匂い。キミの残像。 ft. Kanae Asaba, diff: a...no adjoin
title: おおきなこえで, diff: n...no adjoin
title: おおきなこえで, diff: h...no adjoin
title: おおきなこえで, diff: a...no adjoin
title: おおきなこえで, diff: l...no adjoin
title: 朧, diff: n...no adjoin
title: 朧, diff: h...no adjoin
title: 朧, diff: a...no adjoin
title: 朧, diff: l...no adjoin
title: オドループ, diff: n...no adjoin
title: オドループ, diff: h...no adjoin
title: オドループ, diff: a...no adjoin
title: オドループ, diff: l...no adjoin
title: 踊, diff: n...no adjoin
title: 踊, diff: h...no adjoin
title: 踊, diff: a...no adjoin
title: 踊, diff: l...no adjoin
title: お命ちょうDAI！901娘, diff: n...no adjoin
title: お命ちょうDAI！901娘, diff: h...no adjoin
title: お命ちょうDAI！901娘, diff: a...no adjoin
title: おーまい！らぶりー！すうぃーてぃ！だーりん！, diff: n...no adjoin
title: おーまい！らぶりー！すうぃーてぃ！だーりん！, diff: h...no adjoin
title: おーまい！らぶりー！すうぃーてぃ！だーりん！, diff: a...no adjoin
title: お願いアインシュタイン, diff: n...no adjoin
title: お願いアインシュタイン, diff: h...no adjoin
title: お願いアインシュタイン, diff: a...no adjoin
title: 音楽, diff: n...no a

...

title: TECHNOPHOBIA, diff: n...no adjoin
title: TECHNOPHOBIA, diff: h...no adjoin
title: TECHNOPHOBIA, diff: a...no adjoin
title: THE DEEP STRIKER, diff: n...no adjoin
title: THE DEEP STRIKER, diff: h...no adjoin
title: THE DEEP STRIKER, diff: a...no adjoin
title: THE DEEP STRIKER, diff: l...no adjoin
title: Technicolour Dreams, diff: n...no adjoin
title: Technicolour Dreams, diff: h...no adjoin
title: Technicolour Dreams, diff: a...no adjoin
title: TECHNO Style Essentials, diff: n...no adjoin
title: TECHNO Style Essentials, diff: h...no adjoin
title: TECHNO Style Essentials, diff: a...no adjoin
title: Tell Me More…, diff: n...no adjoin
title: Tell Me More…, diff: h...no adjoin
title: Tell Me More…, diff: a...no adjoin
title: Tell me what you wish feat.らっぷびと, diff: n...no adjoin
title: Tell me what you wish feat.らっぷびと, diff: h...no adjoin
title: Tell me what you wish feat.らっぷびと, diff: a...no adjoin
title: TEXTURE, diff: n...no adjoin
title: TEXTURE, diff: h...no adjoin
title: TEXTURE, 

title: ToyCube Pf.(RX-Ver.S.P.L.), diff: n...no adjoin
title: ToyCube Pf.(RX-Ver.S.P.L.), diff: h...no adjoin
title: ToyCube Pf.(RX-Ver.S.P.L.), diff: a...no adjoin
title: Tp-RZ, diff: n...no adjoin
title: Tp-RZ, diff: h...no adjoin
title: Tp-RZ, diff: a...no adjoin
title: THE PEERLESS UNDER HEAVEN, diff: n...no adjoin
title: THE PEERLESS UNDER HEAVEN, diff: h...no adjoin
title: THE PEERLESS UNDER HEAVEN, diff: a...no adjoin
title: Timepiece phase II, diff: n...no adjoin
title: Timepiece phase II, diff: h...no adjoin
title: Timepiece phase II, diff: a...no adjoin
title: Timepiece phase II (CN Ver.), diff: n...no adjoin
title: Timepiece phase II (CN Ver.), diff: h...no adjoin
title: Timepiece phase II (CN Ver.), diff: a...no adjoin
title: THE SHINING POLARIS, diff: n...has adjoin !!!
title: THE SHINING POLARIS, diff: h...has adjoin !!!
title: THE SHINING POLARIS, diff: n...has adjoin !!!
title: THE SHINING POLARIS, diff: h...has adjoin !!!
title: THE SHINING POLARIS, diff: n...has adjoi

title: Under the Sky, diff: n...no adjoin
title: Under the Sky, diff: h...no adjoin
title: Under the Sky, diff: a...no adjoin
title: UNDERWORLD HOLOGRAPHY, diff: n...no adjoin
title: UNDERWORLD HOLOGRAPHY, diff: h...no adjoin
title: UNDERWORLD HOLOGRAPHY, diff: a...no adjoin
title: Unicorn tail, diff: n...no adjoin
title: Unicorn tail, diff: h...no adjoin
title: Unicorn tail, diff: a...no adjoin
title: UNITE THE HEART, diff: n...no adjoin
title: UNITE THE HEART, diff: h...no adjoin
title: UNITE THE HEART, diff: a...no adjoin
title: Urban Constellations, diff: n...no adjoin
title: Urban Constellations, diff: h...no adjoin
title: Urban Constellations, diff: a...no adjoin
title: U.S.A., diff: n...no adjoin
title: U.S.A., diff: h...no adjoin
title: U.S.A., diff: a...no adjoin
title: Usual Days-remix, diff: n...has adjoin !!!
title: Usual Days-remix, diff: h...has adjoin !!!
title: Usual Days-remix, diff: a...has adjoin !!!
title: V<font size=-2>2</font>, diff: n...no adjoin
title: V<font s

title: waxing and wanding, diff: h...no adjoin
title: waxing and wanding, diff: a...no adjoin
title: waxing and wanding, diff: l...no adjoin
title: Wonderful Escape, diff: n...no adjoin
title: Wonderful Escape, diff: h...no adjoin
title: Wonderful Escape, diff: a...no adjoin
title: We are Disっ娘よっつ打ち命, diff: n...no adjoin
title: We are Disっ娘よっつ打ち命, diff: h...no adjoin
title: We are Disっ娘よっつ打ち命, diff: a...has adjoin !!!
title: Welcome, diff: n...no adjoin
title: Welcome, diff: h...no adjoin
title: Welcome, diff: a...no adjoin
title: Welcome, diff: l...no adjoin
title: WE LOVE SHONAN, diff: n...no adjoin
title: WE LOVE SHONAN, diff: h...no adjoin
title: WE LOVE SHONAN, diff: a...no adjoin
title: We're so Happy (P*Light Remix) IIDX ver., diff: n...no adjoin
title: We're so Happy (P*Light Remix) IIDX ver., diff: h...no adjoin
title: We're so Happy (P*Light Remix) IIDX ver., diff: a...no adjoin
title: WHA, diff: n...no adjoin
title: WHA, diff: h...no adjoin
title: WHAT'S NEXT?, diff: n...no 

title: ZENDEGI DANCE, diff: h...no adjoin
title: ZENDEGI DANCE, diff: a...no adjoin
title: Zenith, diff: n...no adjoin
title: Zenith, diff: h...no adjoin
title: Zenith, diff: a...no adjoin
title: ZEИITH, diff: n...no adjoin
title: ZEИITH, diff: h...no adjoin
title: ZEИITH, diff: a...no adjoin
title: Zenius -I- vanisher, diff: n...no adjoin
title: Zenius -I- vanisher, diff: h...no adjoin
title: Zenius -I- vanisher, diff: a...no adjoin
title: ZEPHYRANTHES, diff: n...no adjoin
title: ZEPHYRANTHES, diff: h...no adjoin
title: ZEPHYRANTHES, diff: a...no adjoin
title: ZEPHYRANTHES, diff: l...no adjoin
title: ZEROIN ON THE LIGHT, diff: n...no adjoin
title: ZEROIN ON THE LIGHT, diff: h...no adjoin
title: ZEROIN ON THE LIGHT, diff: a...no adjoin
title: ZERO-ONE, diff: n...no adjoin
title: ZERO-ONE, diff: h...no adjoin
title: ZERO-ONE, diff: a...no adjoin
title: ZERO-ONE, diff: l...no adjoin
title: ZETA 〜素数の世界と超越者〜, diff: n...no adjoin
title: ZETA 〜素数の世界と超越者〜, diff: h...no adjoin
title: ZETA 〜素数の

In [7]:
with open("adjoins.json", "w") as f:
    json.dump(adjoin_scores, f, indent=2, ensure_ascii=False)

In [6]:
with open("adjoins.json", "r") as f:
    adjoin_scores = json.load(f)

In [8]:
def json2table(adjoin_keys, judge):
    side_header = '<th class=sc_count rowspan="{num_adjoin}", >{side}</th>'
    adjoin_ph = '<td class=sc_count><span class={key_class}>{key}</span></td> <td class=sc_count>{times}回</td>'
    key_classes = {"muri": "muri_num_class", "adjoin": "adjoin_num_class"}

    if not ("隣接皿" in judge or "無理皿" in judge):
        return "<span class=mucha>無茶皿</span>"
    html_tbl = "<details> <summary>{summary}</summary> <table class=sc_count>".format(summary=judgeColoring(judge))
    for side in adjoin_keys:
        for idx, key in enumerate(adjoin_keys[side]):
            html_tbl += "<tr>"
            if idx == 0:
                num_adjoin = len(adjoin_keys[side].keys())
                html_tbl += side_header.format(num_adjoin=num_adjoin, side=side)
            classname = "adjoin_num_class" if adjoincheck(key, side) else "muri_num_class"
            html_tbl += adjoin_ph.format(key_class=classname, key=key, times=adjoin_keys[side][key])
            html_tbl += "</tr>"
    html_tbl += "</table></details>"
    return html_tbl

def char2diff(char_diff):
    index = {"n":"<span class=nml>Normal</span>", 
             "h":"<span class=hpr>Hyper</span>", 
             "a":"<span class=ant>Another</span>", 
             "l":"<span class=leg>Leggendaria</span>"}
    return index[char_diff]

def judgeColoring(judge):
    _ret = ""
    for judge_sp in judge.split(", "):
        for det_char, class_name in zip(["無理皿", "無茶皿", "隣接皿"], ["muri", "mucha", "adjoin"]):
            if det_char in judge_sp:
                _ret += "<span class={class_name}>{judge}</span>".format(class_name=class_name, judge=judge_sp)
    return _ret

def adjoincheck(key, side):
    if side == "1p":
        for keyIdx in range(keyNum - adjoin_key_idx, keyNum+1):
            if str(keyIdx) in key: return False
    elif side == "2p":
        for keyIdx in range(1, keyNum - adjoin_key_idx+1):
            if str(keyIdx) in key: return False
    return True

In [19]:
char2diffstr = {"n":"N", "h":"H", "a":"A", "l":"X"}

header = ["Title", "Difficulty", "Lv", "Type"]
table = \
"""<table><tr class="header"><th class="header version sortable">Ver</th> <th class="header title sortable">Title</th> <th class="header diff sortable">Difficulty</th> <th class="header lv sortable">☆</th> <th class="header type sortable">Type</th></tr>"""
df = pd.DataFrame(adjoin_scores)
df["for_sort"] = df["title"].str.lower()
df_sorted = df.sort_values(["acin", "for_sort"], ascending=[False, True])
for idx, info in df_sorted.iterrows():
    href= "http://textage.cc/score/{ver}/{fbasename}.html?D{difficulity}{lv}00".format(ver=info["ver"], fbasename=info["fbasename"], difficulity=char2diffstr[info["diff"]], lv=hex(info["level"])[-1].capitalize())
    title = info["title"] + " " + info["subtitle"] if info["subtitle"] else info["title"]
    title = "<a href='{href}' target='_blank' rel='noopener noreferrer'>".format(href=href) + title + "</a>"
    data = [info["ver"], title, char2diff(info["diff"]), info["level"], json2table(info["adjoin_keys"], info["judge"])]
    row = "<tr>" if info["acin"] else "<tr class=del>"
    for cell_data in data:
        row += "<td>{cell}</td>".format(cell=cell_data)
    row += "</tr>"
    table += row
table += "</table>"
table.replace("\'", "&#39;")


'<table><tr class="header"><th class="header version sortable">Ver</th> <th class="header title sortable">Title</th> <th class="header diff sortable">Difficulty</th> <th class="header lv sortable">☆</th> <th class="header type sortable">Type</th></tr><tr><td>12</td><td><a href=&#39;http://textage.cc/score/12/100mnm_g.html?DAB00&#39; target=&#39;_blank&#39; rel=&#39;noopener noreferrer&#39;>100% minimoo-G</a></td><td><span class=ant>Another</span></td><td>11</td><td><details> <summary><span class=adjoin>隣接皿</span></summary> <table class=sc_count><tr><th class=sc_count rowspan="1", >1p</th><td class=sc_count><span class=adjoin_num_class>1</span></td> <td class=sc_count>1回</td></tr></table></details></td></tr><tr><td>6</td><td><a href=&#39;http://textage.cc/score/6/19novem.html?DN500&#39; target=&#39;_blank&#39; rel=&#39;noopener noreferrer&#39;>19,November</a></td><td><span class=nml>Normal</span></td><td>5</td><td><details> <summary><span class=adjoin>隣接皿</span></summary> <table class=s

In [20]:
with open("modules/empty.html", "r", encoding="utf-8") as f:
    htmls = f.read()
head_btime, head_atime, tail = htmls.split("<!-- python_split -->")
head = head_btime + datetime.date.today().strftime("%Y.%m.%d") + head_atime
with open("index.html", "w", encoding="utf-8") as f:
    f.write("{head}\n{table}\n{tail}".format(head=head, table=table, tail=tail))

In [95]:
df_sorted.to_csv("test.csv")

In [24]:
np.arange(500).reshape((25,-1)).take(range(5, 25), axis=0).shape

(20, 20)

In [25]:
np.arange(500).reshape((25,-1)).shape

(25, 20)